
## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/tips.csv"
file_type = "csv"

df = spark.read.load(file_location, format='csv',sep=',',inferSchema=True, header=True)

In [0]:
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [0]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [0]:
#only total_bill is dependent variable others all are independent variables
#Now we need convert the categorical features into numerical values for our machine to understand them.
#Handling Categorical Features
from pyspark.ml.feature import StringIndexer


In [0]:
df.columns

Out[7]: ['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [0]:
indexer=StringIndexer(inputCols=['sex', 'smoker', 'day', 'time'],
                      outputCols=['sex_i', 'smoker_i', 'day_i', 'time_i'])

In [0]:
df_i=indexer.fit(df).transform(df)

In [0]:
df_i.show()

+----------+----+------+------+---+------+----+-----+--------+-----+------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_i|smoker_i|day_i|time_i|
+----------+----+------+------+---+------+----+-----+--------+-----+------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|  1.0|     0.0|  1.0|   0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|  0.0|     0.0|  1.0|   0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|  0.0|     0.0|  1.0|   0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|  0.0|     0.0|  1.0|   0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|  1.0|     0.0|  1.0|   0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|  0.0|     0.0|  1.0|   0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|  0.0|     0.0|  1.0|   0.0|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|  0.0|     0.0|  1.0|   0.0|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|  0.0|     0.0|  1.0|   0.0|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|  0.0|     0.0|  1.0|   0.0|
|     10.27|

In [0]:
#grouping independent features using vectorassembler
from pyspark.ml.feature import VectorAssembler

In [0]:
df_i.columns

Out[17]: ['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'sex_i',
 'smoker_i',
 'day_i',
 'time_i']

In [0]:
assembler = VectorAssembler(inputCols=['tip','size','sex_i','smoker_i','day_i','time_i'],
                            outputCol='Independent features')

In [0]:
output = assembler.transform(df_i)

In [0]:
output.show()

+----------+----+------+------+---+------+----+-----+--------+-----+------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_i|smoker_i|day_i|time_i|Independent features|
+----------+----+------+------+---+------+----+-----+--------+-----+------+--------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|  1.0|     0.0|  1.0|   0.0|[1.01,2.0,1.0,0.0...|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|  0.0|     0.0|  1.0|   0.0|[1.66,3.0,0.0,0.0...|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|  0.0|     0.0|  1.0|   0.0|[3.5,3.0,0.0,0.0,...|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|  0.0|     0.0|  1.0|   0.0|[3.31,2.0,0.0,0.0...|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|  1.0|     0.0|  1.0|   0.0|[3.61,4.0,1.0,0.0...|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|  0.0|     0.0|  1.0|   0.0|[4.71,4.0,0.0,0.0...|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|  0.0|     0.0|  1.0|   0.0|[2.0,2.0,0.0,0.0,...|
|     26.88|3.12|  Male|    No

In [0]:
finalized_data= output.select('Independent features','total_bill')

In [0]:
finalized_data.show()

+--------------------+----------+
|Independent features|total_bill|
+--------------------+----------+
|[1.01,2.0,1.0,0.0...|     16.99|
|[1.66,3.0,0.0,0.0...|     10.34|
|[3.5,3.0,0.0,0.0,...|     21.01|
|[3.31,2.0,0.0,0.0...|     23.68|
|[3.61,4.0,1.0,0.0...|     24.59|
|[4.71,4.0,0.0,0.0...|     25.29|
|[2.0,2.0,0.0,0.0,...|      8.77|
|[3.12,4.0,0.0,0.0...|     26.88|
|[1.96,2.0,0.0,0.0...|     15.04|
|[3.23,2.0,0.0,0.0...|     14.78|
|[1.71,2.0,0.0,0.0...|     10.27|
|[5.0,4.0,1.0,0.0,...|     35.26|
|[1.57,2.0,0.0,0.0...|     15.42|
|[3.0,4.0,0.0,0.0,...|     18.43|
|[3.02,2.0,1.0,0.0...|     14.83|
|[3.92,2.0,0.0,0.0...|     21.58|
|[1.67,3.0,1.0,0.0...|     10.33|
|[3.71,3.0,0.0,0.0...|     16.29|
|[3.5,3.0,1.0,0.0,...|     16.97|
|(6,[0,1],[3.35,3.0])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
finalized_data.withColumn('total_bill', finalized_data['total_bill'].cast('double'))

Out[36]: DataFrame[Independent features: vector, total_bill: double]

In [0]:
##train test split
train_data,test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='Independent features',labelCol='total_bill')
regressor = regressor.fit(train_data)

In [0]:
finalized_data.printSchema()

root
 |-- Independent features: vector (nullable = true)
 |-- total_bill: double (nullable = true)



In [0]:
regressor.coefficients

Out[39]: DenseVector([3.115, 3.7364, -0.8072, 1.6288, -0.0616, -0.3566])

In [0]:
regressor.intercept

Out[40]: 0.5901353064568451

In [0]:
#evaluate the test data and giving the predictions
pred_results= regressor.evaluate(test_data)

In [0]:
pred_results.predictions.show()
#final comparison

+--------------------+----------+------------------+
|Independent features|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,1],[2.24,3.0])|     16.04|18.776905468126444|
| (6,[0,1],[2.5,4.0])|     18.35|23.323191896470263|
| (6,[0,1],[3.0,2.0])|      14.0|17.407926638529823|
| (6,[0,1],[4.3,2.0])|      21.7|21.457436323207624|
|[1.1,2.0,1.0,1.0,...|      12.9|12.311036003096138|
|[1.48,2.0,0.0,0.0...|      8.52|12.193372208455356|
|[1.5,2.0,1.0,0.0,...|     10.65|11.448490114305766|
|[1.73,2.0,0.0,0.0...|      9.78|12.972124070893395|
|[1.83,1.0,1.0,0.0...|     10.07| 8.740058081315718|
|[1.92,1.0,0.0,1.0...|      8.58|11.394808818553479|
|[2.0,2.0,1.0,0.0,...|     10.33|13.005993839181842|
|[2.0,2.0,1.0,1.0,...|     27.18|15.114542707873076|
|[2.0,2.0,1.0,1.0,...|      13.0|14.634799601421882|
|[2.0,2.0,1.0,1.0,...|      13.0|14.634799601421882|
|[2.0,2.0,1.0,1.0,...|     10.09|14.573211662797275|
|[2.0,5.0,0.0,1.0,...|     30.46|27.0692904866